# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='/home/insidehome/2nd-PyCrawlerMarathon/homework/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    if new_height == last_height:
        break;
    last_height = new_hight
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 蘋果後市看好！六大投資機構上修Q3營運預期　Q2表現激勵投資信心

▲ 蘋果第二季財報表現佳。（圖／路透社）
記者李瑞瑾／綜合報導
蘋果在日前發布2020第二季度營運表現，由於公認的不確定和動蕩，蘋果沒有提供第三季度的展望。而各家機構分析師則先後發表了對蘋果第三財季及未來的展望，因為蘋果Q2優於預期的表現，使不少機構分析師紛紛調升對蘋果的預期。
蘋果公佈2020會計年度Q2的季度營收為583億美元，即使在新冠病毒大流行期間造成了財務影響，蘋果營收表現還是增長了1％。雖然這個數字遠遠低於其最初的預期目標（630億美元至670億美元），但該公司早在2月就警告投資者，由於新冠肺炎（COVID-19）疫情，它可能會失去達成這一目標的機會。
不出所料，iPhone銷售收入在第二季度確實有所下降，從去年同期的310億美元下降至289.6億美元。但是可穿戴設備和服務，以高於預期的速度彌補了這一差額。
外媒appleinsider整理六大國際投資機構對蘋果後市表現預測，因蘋果第二季度的業績優於許多分析師在4月30日財報電話會議之前的預期，因此也有不少機構提高了對蘋果第三季甚至是全年的預測。
摩根大通（JP Morgan）
首席分析師Samik Chatterjee認為，基於多個因素，蘋果公司的表現將超越大盤。其中包括2019年iPhone的銷售優於預期；4月份產品銷售收入因iPhone SE的推出大幅改善；對iPad和Mac的成長被低估了，因為在家上班的新工作型態崛起；最後則是蘋果在服務業務的成長彈性也比預期更好。
▲ 新工作型態轉變將讓產品銷售有所成長。（圖／路透）
根據蘋果公司的長期預測，Chatterjee將iPhone的第三季度的銷售量，從先前的1500萬部，提高到了2200萬部。
Chatterjee還預計2020年下半年，蘋果5G iPhone會有上行空間，高通對5G需求的最新預測以及供應鏈正常化的信心，也刺激了他對蘋果iPhone 12銷售樂觀看待。
即便蘋果沒有發布第三季度展望，JP摩根將其預估值和12月20日的目標價提高至350美元，高於該公司先前的預測335美元。
科文（Cowen）
由於蘋果公佈的業績優於市場預

▲國稅局受理報稅櫃臺，除以塑膠布與民眾隔開外，部分承辦人員甚至戴上防護面罩防疫。（圖／記者湯興漢攝）
記者李錦奇／台北報導
報稅月來臨，今年因為有疫情影響，財政部也祭出史上最高規格的防疫規定，例如民眾要先量體溫、戴口罩才能進入國稅局，此外，報稅現場也做了座位拉開距離、塑膠布隔離等防疫措施，甚至，部分國稅局人員還戴上防疫面罩，都是史上首見畫面。
現在就跟著《ETtoday新聞雲》記者的鏡頭，一起來看看這些過往未曾見到的畫面，一起見證政府與國人如何因應疫情，完成年度報稅大事。
▲財政部台北國稅局，今年特別在戶外搭棚，讓民眾報稅，避免進入國稅局大樓，帶來防疫風險，這也是史上頭一遭。（圖／記者湯興漢攝）
▲▼台北國稅局高規格防疫。承辦人員以塑膠布與報稅民眾隔開。（圖／記者湯興漢攝）
▲國稅局部分承辦人員戴上防護面罩，手上也戴上簡易手套，小心防疫。（圖／記者湯興漢攝）
▲▼國稅局也在入口處宣導，請民眾保持社交距離，座位區也拉開距離防疫。（圖／記者湯興漢攝）
▲綜所稅申報第一天，財政部長蘇建榮（右三）一早便前往台北國稅局視察，全程戴著口罩，也是史上首見場面。（圖／記者湯興漢攝）
----------------------------------------------------------------------

[11] 功成身退！5月中起　台酒防疫酒精不再強制配送藥局　民眾不要擠藥局囉！

▲台酒防疫酒精。（圖／記者吳靜君攝）
記者吳靜君／台北報導
隨著新冠肺炎（COVID-19）疫情趨緩，為了減輕藥局的負擔，台酒表示，5月12日後，將不再強制配送台酒75度防疫酒精到健保藥局；也就是說，民眾在5月中之後到藥局就可能買不到台酒的防疫酒精。
台酒表示，隨著疫情趨緩，疫情指揮中心已經逐步降低健保藥局的負擔，包含藥局賣口罩鬆綁為週日不配送，藥局可以不營業賣口罩；此外，疫情中心發函給藥師公會、藥劑師公會、台酒公司及中華郵政公司等單位，通知「台酒防疫酒精於下週（5月4日到11日）當週配送最後一批之後，將不再強制配送」。
台酒表示，原本台酒的75度防疫酒精在健保藥局、四大超商（統一超商、全家、萊爾富、OK便利商店）、全聯、美聯社、家樂福、愛買等地方上架銷售，民眾可以就近購買。
但是5月中之後，再送最後一批酒精，6000家健保藥局在就可以不用配合防疫中心販售防疫酒精，台酒防疫酒銷售據

▲小規模營業人再鬆綁，受惠小老闆增加。（圖／行政院提供）
記者吳靜君／台北報導
行政院今（3）日「1988疑難雜症回答part2- 融資篇」記者會，宣布小規模營業人兩大鬆綁條件，其一是將易評分表的核貸分數從70分降為63分，根據行政院估算，原本小規模營業人為50萬，可以再擴大為90萬，再加上核貸成功率提升，受惠的小老闆可望翻倍。
其次是原本並不符合小規模營業人條件，也就是月營業額超過20萬元、必須要開立統一發票的營業人，因為疫情的緣故，所以月營業額下降到20萬元，所以客戶提出401報表的話，也可以申請小規模營業人紓困貸款。
小規模營業人紓困貸款市政府針對月營業額在20萬元以下、「免用統一發票」的小商家推出快速紓困貸款專案，申請貸款50萬元以下、核貸只需3天，利率不超過1%。
這兩項鬆綁，將使央行的小規模營業人紓困貸款可以照顧更多店家和商販。
第二部分是將有開統一發票的店家，但因疫情使月營業額下滑到20萬元的小店家也納入的話，小規模營業人的母數就會擴大為90萬人，再計算貸款成功率來計算，受惠的小老闆可望翻倍。
----------------------------------------------------------------------

[21] 連六天0確診進入振興階段？　經濟部次長林全能：指揮中心宣布就即刻啟動

記者李瑞瑾／台北報導
中央流行疫情指揮中心今（1日）再度宣布零確診，是台灣連續6天都沒有新增案例，而是否台灣能也從「紓困」階段轉到「振興」階段，經濟部次長林全能指出，振興方案已箭在弦上，「只要時間一到就能啟動」。
▲ 5月1日紓困振興記者會。（圖／行政院提供）
林全能表示，在振興面向有很多元的工作，像是要去進行商圈、夜市行銷，林全能指出，目前行銷協助計畫已經開始接受申請，也開始進行審查，希望透過擴大去行銷去刺激復甦。
另外針對先前熱烈討論的振興抵用券，林全能表示，當時就規劃得很具體，相關行政作業也在進行中，「只要指揮中心告知可以啟動，就會啟動。」
另外在酷碰券的部分，林全能指出，目前都在聆聽各方意見，希望在聆聽多元意見之後做最完整的規劃，只要時間一到，就會無縫接軌推出。
----------------------------------------------------------------------

[22] 影／紓困貸款方案多

▲Switch一開賣就秒殺。（圖／翻攝自ptt）
記者陳心怡／綜合報導
任天堂Switch《動物之森》遊戲近期已成為「國民遊戲」，加上疫情的關係，導致銷量大增甚至長期缺貨，讓不少玩家一機難求。而今早PChome 24H、Yahoo 購物中心好不容易有最新一批到貨的任天堂 Switch 主機，但一上架立刻搶購一空，網友哀號根本就搶不到。
PChome 24H和Yahoo 購物中心都選在連假第一天開賣全新的 Switch 主機，其中包含廣受好評的動物森友會聯名版、Switch Lite和健身環大冒險。
不過，開賣沒多久， PTT 網友哀號遍野，紛紛表示九點 PChone 24H 動物森友會聯名版 Switch 一開賣，網站隨即陷入 LAG、甚至出現整個掛掉的情況。網友表示，「一堆殭屍瘋狂的在F5」、「選到加入購物車，然後點下去完全沒動作」、「騙我早起 哭暈」。
Yahoo購物中心十點開賣Switch動物森友會特別版主機，也是一樣的狀況，甚至出現「be right back」，整個賣場都進不去。
記者九點守在電腦前準備開搶，但也完全搶不到，想要原價入手的民眾，恐怕得再等下一批了。
----------------------------------------------------------------------

[32] 勞退新制基金Q1慘虧2823億元　勞工退休帳戶每人平均倒貼2萬3992元！

記者余弦妙／台北報導
勞動部勞動基金運用局今（1）日公布勞動基金最新績效，其中，攸關勞工個人退休金帳戶的新制勞退基金，截至3月底收益數大幅減少2823.9億元，收益率-11.37％，以目前新制共計1,177萬名勞工來計算，每名勞工退休帳戶平均減少約2萬3992元。
這個意思是，如果虧損情況沒有改變，等到今年底結算時，每位勞工的退休金帳戶，會被倒扣虧損金額，好比投資失敗，投資人要認列損失，分紅不成反而倒貼。
勞保局補充，這是勞工退休，才領得到的退休金，每年只是先做損益計算，分攤到每個勞工退休帳戶裡。
至於整體的勞動基金，截至3月收益數慘賠4712.5億元，收益率為負11.16％；其中攸關勞工個人退休金帳戶的新制勞退基金，減少2823.9億元，收益率為-11.37％。
勞動基金局表示，整體勞動基金截至3月底止規模為4兆2,852 億元，其中，新制勞退基金規模為2兆4,97

▲大江生醫董事長林詠翔（左）日前聯袂衛福部政務次長何啟功（右）發表快速病毒掃描機QVS-96。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
大江生醫（8436）公布今年首季毛利44%，稅後純益3.76億元，年減39.65%，每股稅後純益（EPS）3.09元，同時營業費用大幅減少近1.4億元，帶動淨利率較去年第4季成長17%。大江表示，第2季營收預期將會更好。
大江宣布，經過董事會通過決議，將配發現金股利8.88元，以4月30日收盤價230.5元計算，殖利率達3.86% ，無論是獲利表現及發展前景來看，預期將成為投資者關注的市場焦點。
大江表示，受惠於免疫產品熱銷，毛利率創下一年來新高，不受疫情影響，單季毛利率逆勢成長7.3% ，雖然各地因新冠肺炎疫情導致經濟活動大幅減少，甚至還有封城等影響原料價成品流通等不利條件，但大江在去年底就提前開始布局。
大江指出，公司在去年底就先行進口大量免疫產品生產物資，並為品牌客戶安排成品物流服務，協助品牌在封城與不景氣劣勢中，仍持續營運創造營收，更把握宅經濟契機，為品牌客戶提供「華麗直播」服務，曾創下短短2小時內帶貨銷售1500萬人民幣的佳績。
大江說明，日前聯袂衛福部發表的快速病毒掃描機QVS-96，已經疾管局驗證確效發證，將進入昆陽實驗室上線服役，成為近期新冠病毒檢疫新品中，第一個正式應用在檢疫實務應用的產品，並將QVS-96單日檢疫樣品數提升達2016件，等同於二台QVS-96S，單日檢測量即超越目前全台34家檢驗所處理量。
大江強調，目前正著手與各國疾管機關接洽，迅速在機場及大型企業佈建高精準、高通量、低成本的病毒檢疫設備，將台灣斐然的防疫經驗，拓展至全球各地，協助恢復日常生活與經濟發展。
----------------------------------------------------------------------

[43] 疫情助攻！蘋果iMessage和FaceTime　每日使用人次創歷史新高　

▲ 疫情讓FaceTime使用人次創紀錄。（圖／達志影像／美聯社）
記者李瑞瑾／綜合報導
蘋果首席執行長提姆．庫克（Tim Cook）在2020年第二季度的財報電話會議上表示，iMessage和FaceTime在新冠病毒（COVID-19）大流行期間，用戶每天使用次數都創下了新的歷史記錄。
外媒9to5mac指出，